# Project1 - Group 4


### Library dependencies

In [171]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import requests
# import time
# from scipy.stats import linregress
# import datetime

from os import listdir
from os.path import isfile, join
# Reference: https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory

# Data sources - load, consolidate, and merge

In [172]:
# Paths to analysis data files:
# Uber trips files from Kaggle
uber2014_path = join("data", "kaggle_csv", "uber-raw-data-2014")
# New York city Weather for months corresponding to the Uber trips
noaa2014_path = "data/noaa_csv/2014"

### Consolidate Uber trips data

In [173]:
# Read the Uber trips data and consolidate into one file
# Uber trips files from Kaggle
uber2014 = pd.DataFrame()
files = [join(uber2014_path, f) for f in listdir(uber2014_path) if isfile(join(uber2014_path, f))]
for f in files:
    data = pd.read_csv(f)
    if (len(uber2014)) == 0:    
        uber2014 = data.copy(deep=True)
    else:
        combined = pd.concat([uber2014, data], axis=0, ignore_index=True)
        uber2014 = combined.copy(deep=True)

In [174]:
# Check for and remove duplicate rows
uber2014 = uber2014.drop_duplicates()

In [175]:
# Convert from US-format date, and split orginal datetime into Date and Time columns
uber2014["DateTime"] = pd.to_datetime(uber2014["Date/Time"], format='%m/%d/%Y %H:%M:%S')
uber2014["Date"] = uber2014["DateTime"].dt.normalize()
uber2014 = uber2014[["Date", "DateTime", "Lat", "Lon", "Base"]]

In [176]:
# Write out the combined dataset
uber2014.to_csv(join("data", "combined", "uber-combined-data-2014.csv"), encoding = "UTF-8", index=False)
print(f"Total unique rows (Uber rides 2014 data): {len(uber2014)}")

Total unique rows (Uber rides 2014 data): 4451746


### Consolidate New York City weather data for the trips period (& convert to metric units)

In [177]:
# Formulas for conversion of temperature and rainfall to metric units
def Convert_Fahrenheit_to_Celsius(deg_F):
    deg_C = (deg_F - 32) / (9/5)
    return deg_C

def Inches_to_Millimetres(inches):
    millimetres = inches * 25.4
    return millimetres

In [178]:
# Read the Weather data, convert to metric units, and consolidate into one file
# Uber trips files from NOAA
weather2014 = pd.DataFrame()
files = [join(noaa2014_path, f) for f in listdir(noaa2014_path) if isfile(join(noaa2014_path, f))]
for f in files:
    data = pd.read_csv(f)
    if (len(weather2014)) == 0:    
        weather2014 = data.copy(deep=True)
    else:
        combined = pd.concat([weather2014, data], axis=0, ignore_index=True)
        weather2014 = combined.copy(deep=True)
        
# Fix column names to remove newlines introduced by 'word wrap' in CSV file header
weather2014.columns = ["Year", "Month", "Day", "MaxTemp_(degF)", "MinTemp_(degF)", "Rain_MeltedSnow_Etc_(in)", "Snow_IcePellets_Hail_(in)", "Snow_IcePellets_Hail_IceOnGround_(in)"]

# We want to keep the 24-hour observation values only, so drop the "Snow_IcePellets_Hail_IceOnGround_(in)" column as that represented 'at observation time' data.
weather2014 = weather2014[["Year", "Month", "Day", "MaxTemp_(degF)", "MinTemp_(degF)", "Rain_MeltedSnow_Etc_(in)", "Snow_IcePellets_Hail_(in)"]]

# Combine Year-Month-Day columns for form a date value in yyyy-mm-dd format
weather2014["Date"] = pd.to_datetime(dict(year=weather2014.Year, month=weather2014.Month, day=weather2014.Day))

# Convert temperature and rainfall to metric units
weather2014["MaxTemp_degC"] = weather2014["MaxTemp_(degF)"].apply(Convert_Fahrenheit_to_Celsius).round(3)
weather2014["MinTemp_degC"] = weather2014["MinTemp_(degF)"].apply(Convert_Fahrenheit_to_Celsius).round(3)
weather2014["Rain_MeltedSnow_Etc_mm"] = weather2014["Rain_MeltedSnow_Etc_(in)"].apply(Inches_to_Millimetres).round(3)
weather2014["Snow_IcePellets_Hail_mm"] = weather2014["Snow_IcePellets_Hail_(in)"].apply(Inches_to_Millimetres).round(3)

# Keep just the metric values
weather2014 = weather2014[["Date", "MaxTemp_degC", "MinTemp_degC", "Rain_MeltedSnow_Etc_mm", "Snow_IcePellets_Hail_mm"]]

# Write out combined data with metric units
weather2014.to_csv(join("data", "combined", "weather-combined-data-metric-2014.csv"), encoding = "UTF-8", index=False)


## Merge Uber trips and Weather data

In [179]:
# Merge the Uber data with the Weather data on the Date column
Uber_Weather_merged_data = pd.merge(uber2014, weather2014, on="Date", how='left')

# Display the first few rows of the merged DataFrame
Uber_Weather_merged_data.head()

,Date,DateTime,Lat,Lon,Base,MaxTemp_degC,MinTemp_degC,Rain_MeltedSnow_Etc_mm,Snow_IcePellets_Hail_mm
0,2014-04-01,2014-04-01 00:11:00,40.7690,-73.9549,B02512,15.556,3.889,0.0,0.0
1,2014-04-01,2014-04-01 00:17:00,40.7267,-74.0345,B02512,15.556,3.889,0.0,0.0
2,2014-04-01,2014-04-01 00:21:00,40.7316,-73.9873,B02512,15.556,3.889,0.0,0.0
3,2014-04-01,2014-04-01 00:28:00,40.7588,-73.9776,B02512,15.556,3.889,0.0,0.0
4,2014-04-01,2014-04-01 00:33:00,40.7594,-73.9722,B02512,15.556,3.889,0.0,0.0


In [180]:
# Cross check the number of rows
num_rows = len(Uber_Weather_merged_data)
print("Number of rows in Uber_Weather_merged_data:", num_rows)

Number of rows in Uber_Weather_merged_data: 4451746


# Exploratory Data Analysis